## Presión dentro de la burbuja

Calculo la presión dentro de la burbuja

### Importo dependencias

In [ ]:
#Código necesario para importar funciones de otros notebooks .ipynb

import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod


class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

sys.meta_path.append(NotebookFinder())

#Ejemplo
# from derivada_reacciones import *

In [ ]:
#Importo notebooks
from funciones_math import Vol
from parametros import var_termo, sigma, mu, rol, a_h2o, b_h2o, Na, Rg
from funciones_qcas import n_tot

### Van-der-Waals

In [ ]:
def ctes_VanderWaals(ngas, nvap, ntot):
   #Calculo las ctes de Van de Waals a partir de las ecs 2.2.2.2 y 2.2.2.3
   #ngas = ng es la cantidad de partículas de gas no condensable
   #nvap = nv es la cantidad de partículas de vapor (h2o)
   #ntot = nt es la cantidad de partículas totales.

   # a_g = a_Ar
   # b_g = b_Ar
   # a_v = a_h2o
   # b_v = b_h2o
   # a_gv = math.sqrt(a_g*a_v) #VW gas-vapor a_{g-v}. Antes se llamaba a_Arh2o
   # b_gv = pow(0.5*(pow(b_g,1.0/3.0)+pow(b_v,1.0/3.0)),3.0) #VW gas-vapor b_{g-v}. Antes se llamaba b_Arh2o
   # a = a_g*(ngas/ntot)**2 + 2.0*a_gv*(ngas/ntot)*(nvap/ntot)+ a_v*(nvap/ntot)*(nvap/ntot) #const a de VW promediada con la cantidad de particulas
   # b = b_g*(ngas/ntot)**2 + 2.0*b_gv*(ngas/ntot)*(nvap/ntot)+ b_v*(nvap/ntot)*(nvap/ntot) #const b de VW promediada con la cantidad de particulas

   return a_h2o,b_h2o

def volumen_molar(V, ntot):
   # Calcula el volumen molar, necesario en el cálculo de la presión del gas empleando la ec. de Van der Waals.
   # V es el volumen total
   # ntot es el nro total de partículas
   # Na es el nro de Avogadro
   return V/ntot*Na #volumen molar

def volumen_molar_deriv():
   # dvdt=-V*Na/(ntot*ntot)*dntdt+4.0*pi*y[1]*y[1]*Na/ntot*y[2];
   # //derivada del volumen molar con respecto a t
   return 0

def ctes_VanderWaals_deriv():
   # dadt=-2.0/pow(ntot,3.0)*(a_Ar*ngas*ngas+2.0*a_Arh2o*ngas*y[Nvar2+6]-a_Arh2o*ngas*ntot+
   # a_h2o*y[Nvar2+6]*y[Nvar2+6]-a_h2o*y[Nvar2+6]*ntot)*dntdt;
   # //derivada respecto del tiempo de la const. a de VW

   # dbdt=-2.0/pow(ntot,3.0)*(b_Ar*ngas*ngas+2.0*b_Arh2o*ngas*y[Nvar2+6]-b_Arh2o*ngas*ntot+
   # b_h2o*y[Nvar2+6]*y[Nvar2+6]-b_h2o*y[Nvar2+6]*ntot)*dntdt;
   # //derivada respecto del tiempo de la const. b de VW

   return 0, 0

### p_g

In [ ]:
from funciones_qcas import n_gas, n_tot

In [ ]:
def p_g(n, Vol, T_bc):

   #Cuentas previas
   ngas = n_gas(n) #Nro de partículas de gas no condensable.
   n_H2O = n[5]
   ntot = n_tot(n)

   #Calculo las constantes de Van der Waals:
   a,b = ctes_VanderWaals(ngas, n_H2O, ntot)
   v = volumen_molar(Vol, ntot) #volumen molar

   #Presion del gas calculada a partir de 2.2.2.1 empleando la ec. de Van der Waals.
   pg=Rg*T_bc/(v-b)-a/(v*v)
   
   return pg

### p_B

In [ ]:
from funciones_qcas import rog

In [ ]:
def p_B(y, n):
    #Presion en la interfase del lado del liquido
    #t: tiempo necesario para calcular y(3)
    #y: conjunto de variables termodinámicas
    #n: número de partículas de cada especia
    #n0: número de partículas de cada especia en el estado inicial
    #nAr: número de partículas de argón. Necesario para calcular mp, aunque no debería serlo

    #Asigno las variables termodinámicas
    R = y[var_termo["R"]]
    v_R = y[var_termo["R"]]
    mp = y[var_termo["mp"]]
    T_bc = y[var_termo["T_bc"]]

    #Calculo la presión del gas dentro de la burbuja (pg)
    pg = p_g(n, Vol(R), T_bc)

    #Calculo la presión pB
    pB=pg-2.0*sigma/R-4.0*mu/R*(v_R-mp/rol)-mp*mp*(1.0/rol-1.0/rog(y))

    return pB

In [ ]:
def dpBdt(y,n):
    #Hacer esta cuenta puede llegar a ser muy complicado. Necesito en el camino usar conservación de la energía. Por ahora no lo implemento 
    return 0